# TODO
* take pixels with no ice thickness and just make the hydropotential the graviational potential

Notebook uses BedMachine Antarctica ice-surface and bed topographies to calculate and export subglacial hydropotential using the Shreve (1972) hydropotential equation.

Written 2025/07/20 by W. Sauthoff (wsauthoff.github.io) and M. R. Siegfried (mrsiegfried.github.io).

# Setup computing environment

In [ ]:
# Import packages
import earthaccess
import fsspec
import geopandas as gpd
import glob
import json
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from mpl_toolkits.axes_grid1 import make_axes_locatable
import numpy as np
import os
from pyproj import CRS
import shutil
import xarray as xr
import zarr
import zipfile

os.makedirs('output', exist_ok = True)

%matplotlib widget

# Define data and script directories dependent on home environment
if os.getenv('HOME') == '/home/jovyan':
    DATA_DIR = '/home/jovyan/data'
    OUTPUT_DIR = '/output'
# DATA_DIR = '~/data'

# Define functions

In [ ]:
def combine_quadrants_from_files(input_data, ds_prefixes, x_dim='x', y_dim='y', keep_vars=None):
    """
    Combine one or more quadrant NetCDF datasets into a single dataset using xarray.
    Automatically detects which quadrants (A1–A4) are available and concatenates accordingly.
    If only one quadrant is provided, it is returned as-is (no trimming or merging).

    Parameters
    ----------
    input_data : str
        Directory path containing the quadrant NetCDF files.
    ds_prefixes : list of str
        List of dataset prefixes corresponding to available quadrants.
        Example: ['CryoSat2_SARIn_delta_h_A1', 'CryoSat2_SARIn_delta_h_A2']
    x_dim, y_dim : str
        Names of the x and y dimensions.
    keep_vars : list, optional
        List of variables to keep (coordinates always preserved).

    Returns
    -------
    xarray.Dataset
        Combined dataset across available quadrants, with overlaps removed if applicable.
    """

    def drop_unwanted_variables(dataset, keep_vars):
        """Drop variables not listed in keep_vars, preserving coordinates."""
        if keep_vars is None:
            return dataset
        coord_vars = set(dataset.coords.keys())
        vars_to_preserve = set(keep_vars) | coord_vars
        vars_to_drop = [v for v in dataset.variables if v not in vars_to_preserve]
        return dataset.drop_vars(vars_to_drop, errors='ignore')

    quadrant_order = {'A1': 0, 'A2': 1, 'A3': 2, 'A4': 3}
    ordered_datasets = [None, None, None, None]

    # Load each available dataset
    for prefix in ds_prefixes:
        found_suffix = next((suffix for suffix in quadrant_order if prefix.endswith(suffix)), None)
        if found_suffix is None:
            raise ValueError(f"Prefix '{prefix}' does not end with a valid quadrant (A1–A4).")

        if not isinstance(input_data, str):
            raise TypeError("input_data must be a directory path (str).")

        pattern = os.path.join(input_data, f"{prefix}.nc")
        files = glob.glob(pattern)
        if not files:
            raise ValueError(f"No file found for pattern: {pattern}")

        ds = xr.open_dataset(files[0])
        ds = drop_unwanted_variables(ds, keep_vars)
        ordered_datasets[quadrant_order[found_suffix]] = ds

    # Filter out missing quadrants
    datasets = [d for d in ordered_datasets if d is not None]
    available_quadrants = [q for q, idx in quadrant_order.items() if ordered_datasets[idx] is not None]
    print(f"→ Found {len(datasets)} quadrant(s): {', '.join(available_quadrants)}")

    # Case 1: Only one quadrant — return as-is (no edge trimming)
    if len(datasets) == 1:
        print("→ Only one quadrant found — skipping trimming and concatenation.")
        return datasets[0]

    # Case 2: Multiple quadrants — apply trimming
    processed = []
    for q, ds in zip(available_quadrants, datasets):
        if q == 'A1':
            processed.append(ds)
        elif q == 'A2':
            processed.append(ds.drop_sel({x_dim: 0}, errors='ignore'))
        elif q == 'A3':
            processed.append(ds.drop_sel({y_dim: 0}, errors='ignore'))
        elif q == 'A4':
            processed.append(ds.drop_sel({x_dim: 0, y_dim: 0}, errors='ignore'))

    # Concatenate depending on which quadrants exist
    if set(available_quadrants) == {'A1', 'A2'}:
        combined = xr.concat([processed[1].isel({x_dim: slice(0, -1)}), processed[0]], dim=x_dim)
    elif set(available_quadrants) == {'A3', 'A4'}:
        combined = xr.concat([processed[0].isel({x_dim: slice(0, -1)}), processed[1]], dim=x_dim)
    elif set(available_quadrants) == {'A1', 'A3'}:
        combined = xr.concat([processed[1].isel({y_dim: slice(0, -1)}), processed[0]], dim=y_dim)
    elif set(available_quadrants) == {'A2', 'A4'}:
        combined = xr.concat([processed[1].isel({y_dim: slice(0, -1)}), processed[0]], dim=y_dim)
    elif len(processed) == 4:
        A12 = xr.concat([processed[1].isel({x_dim: slice(0, -1)}), processed[0]], dim=x_dim)
        A34 = xr.concat([processed[2].isel({x_dim: slice(0, -1)}), processed[3]], dim=x_dim)
        combined = xr.concat([A34.isel({y_dim: slice(0, -1)}), A12], dim=y_dim)
    else:
        raise ValueError(f"Unsupported combination of quadrants: {available_quadrants}")

    print(f"→ Combined dataset shape: {combined[x_dim].size} × {combined[y_dim].size}")
    return combined

In [ ]:
def combine_quadrants_from_files_S3(
    datasets_files,
    ds_prefixes,
    group=None,
    keep_vars=None,
    x_dim='x',
    y_dim='y',
    retries=3
):
    """
    Open, filter, and combine one or more ATL15 quadrant datasets (A1–A4).
    Works with S3 file-like objects. If only one quadrant is provided,
    returns it unmodified (no trimming or concatenation).

    Parameters
    ----------
    datasets_files : list
        List of file-like objects (e.g., S3 paths or fsspec file objects).
    ds_prefixes : list of str
        Dataset prefixes such as ['ATL15_A1', 'ATL15_A2', ...].
    group : str
        NetCDF group to open (default None opens root level).
    keep_vars : list, optional
        Variables to keep; coordinates always preserved.
    x_dim, y_dim : str
        Names of horizontal dimensions.
    retries : int
        Number of retry attempts when opening datasets.

    Returns
    -------
    xarray.Dataset
        Combined dataset if multiple quadrants given; otherwise, a single dataset.
    """

    def drop_unwanted_variables(dataset):
        if keep_vars is None:
            return dataset
        coord_vars = set(dataset.coords.keys())
        vars_to_preserve = set(keep_vars) | coord_vars
        vars_to_drop = [v for v in dataset.variables if v not in vars_to_preserve]
        return dataset.drop_vars(vars_to_drop, errors='ignore')

    def safe_open_and_filter(file, group=group):
        for attempt in range(retries):
            try:
                print(f"Opening file: {file} (Attempt {attempt+1})")
                ds = xr.open_dataset(file, group=group)
                print(f"Successfully opened file: {file}")
                return drop_unwanted_variables(ds)
            except Exception as e:
                print(f"Attempt {attempt+1} failed for {file}: {e}")
                if attempt < retries - 1:
                    print("Retrying...")
                else:
                    raise e

    # --- Load datasets matching prefixes ---
    quadrant_order = {'A1': 0, 'A2': 1, 'A3': 2, 'A4': 3}
    ordered_datasets = [None, None, None, None]

    for prefix in ds_prefixes:
        found_suffix = next((suffix for suffix in quadrant_order if prefix.endswith(suffix)), None)
        if found_suffix is None:
            raise ValueError(f"Prefix '{prefix}' does not end with a valid quadrant (A1–A4).")

        # Match file by substring (works with S3 paths)
        matching_files = [f for f in datasets_files if prefix in str(f)]
        if not matching_files:
            print(f"No file matched for prefix: {prefix}")
            continue

        ds = safe_open_and_filter(matching_files[0], group=group)
        ordered_datasets[quadrant_order[found_suffix]] = ds

    # --- Filter out missing quadrants ---
    datasets = [d for d in ordered_datasets if d is not None]
    available_quadrants = [q for q, idx in quadrant_order.items() if ordered_datasets[idx] is not None]
    print(f"→ Found {len(datasets)} quadrant(s): {', '.join(available_quadrants)}")

    # --- Case 1: Single quadrant → return unmodified ---
    if len(datasets) == 1:
        print("→ Only one quadrant found — skipping trimming and concatenation.")
        return datasets[0]

    # --- Case 2: Multiple quadrants → apply overlap trimming ---
    processed = []
    for q, ds in zip(available_quadrants, datasets):
        if q == 'A1':
            processed.append(ds)
        elif q == 'A2':
            processed.append(ds.drop_sel({x_dim: 0}, errors='ignore'))
        elif q == 'A3':
            processed.append(ds.drop_sel({y_dim: 0}, errors='ignore'))
        elif q == 'A4':
            processed.append(ds.drop_sel({x_dim: 0, y_dim: 0}, errors='ignore'))

    # --- Combine according to which quadrants exist ---
    if set(available_quadrants) == {'A1', 'A2'}:
        combined = xr.concat([processed[1].isel({x_dim: slice(0, -1)}), processed[0]], dim=x_dim)
    elif set(available_quadrants) == {'A3', 'A4'}:
        combined = xr.concat([processed[0].isel({x_dim: slice(0, -1)}), processed[1]], dim=x_dim)
    elif set(available_quadrants) == {'A1', 'A3'}:
        combined = xr.concat([processed[1].isel({y_dim: slice(0, -1)}), processed[0]], dim=y_dim)
    elif set(available_quadrants) == {'A2', 'A4'}:
        combined = xr.concat([processed[1].isel({y_dim: slice(0, -1)}), processed[0]], dim=y_dim)
    elif len(processed) == 4:
        A12 = xr.concat([processed[1].isel({x_dim: slice(0, -1)}), processed[0]], dim=x_dim)
        A34 = xr.concat([processed[2].isel({x_dim: slice(0, -1)}), processed[3]], dim=x_dim)
        combined = xr.concat([A34.isel({y_dim: slice(0, -1)}), A12], dim=y_dim)
    else:
        raise ValueError(f"Unsupported combination of quadrants: {available_quadrants}")

    print(f"→ Combined dataset shape: {combined[x_dim].size} × {combined[y_dim].size}")
    return combined

In [ ]:
def selective_clip(ds, 
                   SARIn_expand_date=np.datetime64('2013-10-01T22:30:00.000000000'),
                   geom_early=None, 
                   geom_late=None
                  ):

    # Dynamic CRS assignment
    # Extract the CRS info (EPSG code or WKT) from the grid mapping variable
    # stored in your dataset.
    try:
        # Try to grab the simple EPSG string first
        crs_info = ds['spatial_ref'].attrs['spatial_ref']
    except KeyError:
        # Fallback to WKT if 'spatial_ref' isn't there
        crs_info = ds['spatial_ref'].attrs['crs_wkt']
        
    # Apply the CRS to the current chunk
    ds.rio.write_crs(crs_info, inplace=True)
    
    if ds.time < SARIn_expand_date:
        return ds.rio.clip(geom_early, drop=False)
    elif SARIn_expand_date <= ds.time:
        return ds.rio.clip(geom_late, drop=False)
    else:
        return ds

# Access data

## Boundaries

In [ ]:
# #TODO: download stationary outline gdf's; must rename the output in Sauthoff-2025-GRL
# # try to get wildcard operator to work: #-w "*.geojson"
# !zenodo_get -o ./input/lake_outlines/Sauthoff_2025_GRL/temp 10.5281/zenodo.15758712

In [ ]:
# #TODO: download stationary outline gdf's; must rename the output in Sauthoff-2025-GRL

# # Import Sauthoff, 2025 (active subglacial lake evolving outlines)
# # https://doi.org/10.5281/zenodo.14963551
# !zenodo_get -o ./input/lake_outlines/Sauthoff_2025_GRL/evolving_outlines 10.5281/zenodo.15758712 -w "Sauthoff-2025-GRL/output/lake_outlines/evolving_outlines/*.geojson"

In [ ]:
# !zenodo_get -l 10.5281/zenodo.15758712

In [ ]:
# #TODO: Maybe try adding evolving_outlines to the file names
# !zenodo_get -o ./input/lake_outlines/Sauthoff_2025_GRL/evolving_outlines 10.5281/zenodo.15758712 -w "*evolving_outlines*.geojson"

In [ ]:
# might not be working because zenodo data is zipped

In [ ]:
# Import CryoSat-2 SARIn mode mask
# repo: https://github.com/wsauthoff/Sauthoff-2025-GRL (https://doi.org/10.5281/zenodo.15758712)
# See 0_preprocess_data.ipynb for data source and pre-processing steps within Sauthoff and others, 202X
#TODO: change to access from one Sauthoff_2025_GRL folder
# gdf_SARIn_3_1 = gpd.read_file('input/CS2_SARIn_mode_masks/gdf_SARIn_3_1.geojson')
# gdf_SARIn_3_6 = gpd.read_file('input/CS2_SARIn_mode_masks/gdf_SARIn_3_6.geojson')


gdf_SARIn_3_1 = gpd.read_file('input/Sauthoff_2025_GRL/gdf_SARIn_3_1.geojson')
gdf_SARIn_3_6 = gpd.read_file('input/Sauthoff_2025_GRL/gdf_SARIn_3_6.geojson')

## BedMachine Antarctica

In [ ]:
# Direct-access stream BedMachine Antarctica bed and surface topography v3 (Morlighem et al., 2020; Morlighem, 2022)
# https://nsidc.org/data/NSIDC-0756
try:
    results = earthaccess.search_data(
        doi='10.5067/FPSU0V1MWUB6',
        cloud_hosted=True,
        # bounding_box=(1, -89, -1, -89)
    )
    
    if not results:
        raise ValueError('No datasets found matching the search criteria')
        
    # Open data granules as s3 files to stream
    files = earthaccess.open(results)
    
    # Check if files list is empty
    if not files:
        raise FileNotFoundError('No files were opened from the search results')
        
    # Check if first file exists/is valid
    if isinstance(files[0], FileNotFoundError):
        raise FileNotFoundError(f'Could not access file: {files[0]}')
        
    # Print file name to ensure expected dataset
    print(f'Attempting to open: {files[0]}')
    
    # Open file into xarray dataset
    bedmachine = xr.open_dataset(files[0], engine='h5netcdf')
    print('Dataset successfully loaded')

except FileNotFoundError as e:
    print(f'File access error: {e}')
except ValueError as e:
    print(f'Search error: {e}')
except Exception as e:
    print(f'Unexpected error: {e}')

'''
Note: "Search error: can only read bytes or file-like objects with engine='scipy' or 'h5netcdf'" seems to indicate a FileNotFoundError,
signaling a problem at the data center.
'''

In [ ]:
# View dataset
bedmachine

In [ ]:
# # Open local copy of file if streaming fails
# bedmachine = xr.open_dataset('/home/jovyan/temp/BedMachineAntarctica-v3.nc')
# bedmachine

## CryoSat-2 SARIn ATL14 DEM and ATL15 delta h

In [ ]:
# # Import Smith and Sauthoff, 2025 (CryoSat-2 SARIn Height Change and Reference DEM for Antarctica)
# # https://doi.org/10.5281/zenodo.14963551
# !zenodo_get -o ~/data 10.5281/zenodo.14963551

In [ ]:
# Import CryoSat-2 SARIn delta_h and data_count dataset

# Specify the variables to keep
keep_vars = ['time', 'y', 'x', 'delta_h', 'data_count', 'polar_stereographic']

# Combine quadrants into one data set
CS2_SARIn_data = combine_quadrants_from_files(DATA_DIR, 
    ds_prefixes=[
                 'CryoSat2_SARIn_delta_h_A1',
                 'CryoSat2_SARIn_delta_h_A2',
                 'CryoSat2_SARIn_delta_h_A3',
                 'CryoSat2_SARIn_delta_h_A4'
                ], 
    keep_vars=keep_vars)

# CRS assignment
# try:
#     # Try to grab the simple EPSG string first
#     crs_info = CS2_SARIn_data['polar_stereographic'].attrs['spatial_ref']
# except KeyError:
#     # Fallback to WKT if 'spatial_ref' isn't there
#     crs_info = CS2_SARIn_data['polar_stereographic'].attrs['crs_wkt']
    
# # Apply the CRS to the current chunk
# CS2_SARIn_data.rio.write_crs(crs_info, inplace=True)

# View data set
CS2_SARIn_data

In [ ]:
# Import CryoSat-2 SARIn DEM dataset

# Specify the variables to keep
keep_vars = ['time', 'y', 'x', 'h']

# Combine quadrants into one data set
CS2_SARIn_DEM = combine_quadrants_from_files(DATA_DIR, 
    ds_prefixes=[
                 'CryoSat2_SARIn_DEM_A1',
                 'CryoSat2_SARIn_DEM_A2',
                 'CryoSat2_SARIn_DEM_A3',
                 'CryoSat2_SARIn_DEM_A4'
                ], 
    keep_vars=keep_vars)

# View data set
CS2_SARIn_DEM

In [ ]:
# plt.close()
# CS2_SARIn_DEM['h'].plot()
# plt.show()

In [ ]:
# Create ICESat-2 absolute heights from ATL14 DEM and ATL15 delta_h data
CS2_h = CS2_SARIn_DEM['h'] + CS2_SARIn_data['delta_h']
CS2_h = CS2_h.rio.write_crs('EPSG:3031') # FIXME: write from crs of DEM or data?

# View newly created dataset
CS2_h

## ICESat-2 ATL14 DEM and ATL15 delta h

In [ ]:
# Find ICESat-2 ATL14 v004 data granules
results = earthaccess.search_data(
    doi='10.5067/ATLAS/ATL14.004',
    bounding_box=(180, -90, -180, -60),  # (lower_left_lon, lower_left_lat , upper_right_lon, upper_right_lat))
    cloud_hosted=True,
)

# Open data granules as s3 files to stream
files = earthaccess.open(results)

In [ ]:
files

In [ ]:
variables_to_keep = ['x', 'y', 'Polar_Stereographic', 'h']

IS2_ATL14 = combine_quadrants_from_files_S3(
    datasets_files=files,
    ds_prefixes=[
        'ATL14_A1',
        'ATL14_A2',
        'ATL14_A3',
        'ATL14_A4'
    ],
    keep_vars=variables_to_keep
)

# View dataset
IS2_ATL14

In [ ]:
# # Authenticate with Earthdata Login
# earthaccess.login()

# Find ICESat-2 ATL15 v004 data granules
results = earthaccess.search_data(
    doi='10.5067/ATLAS/ATL15.004',
    bounding_box=(180, -90, -180, -60),  # (lower_left_lon, lower_left_lat , upper_right_lon, upper_right_lat))
    cloud_hosted=True,
)

# Open data granules as s3 files to stream
files = earthaccess.open(results)

In [ ]:
# Filter to 1 km resolution data sets
# filtered_files = [f for f in files if '01km' in str(f)]
filtered_files = [f for f in files if '01km' in str(f) and 'ATL15' in str(f)]

# Delete intermediary objects for memory conservation
del results, files

# Sort alphabetically by the data set file name
filtered_files.sort(key=lambda x: str(x).split('/')[-1])

# Display filtered list
filtered_files

In [ ]:
variables_to_keep = ['x', 'y', 'time', 'Polar_Stereographic', 'delta_h']

IS2_ATL15 = combine_quadrants_from_files_S3(
    datasets_files=filtered_files,
    ds_prefixes=[
        'ATL15_A1',
        'ATL15_A2',
        'ATL15_A3',
        'ATL15_A4'
    ],
    group='delta_h',
    keep_vars=variables_to_keep
)

# View dataset
IS2_ATL15

In [ ]:
# # Add datasets attributes
# IS2_ATL15.attrs['identifier_product_DOI'] = 'doi:10.5067/ATLAS/ATL15.004'
# IS2_ATL15.attrs['shortName'] = 'ATL15'

# # View data set
# IS2_ATL15

In [ ]:
# Create ICESat-2 absolute heights from ATL14 DEM and ATL15 delta_h data
IS2_h = IS2_ATL14['h'] + IS2_ATL15['delta_h']

# View newly created dataset
IS2_h

In [ ]:
# plt.close()
# IS2_h[:,:,0].plot()
# plt.show()

## Pre-process CryoSat-2

In [ ]:
# Clip to SARIn mode mask to eliminate extrapolated data beyond observations
CS2_h = CS2_h.chunk({'time': 10})
CS2_h_clipped = CS2_h.groupby('time').map(
    lambda x: selective_clip(x,
        geom_early=gdf_SARIn_3_1.geometry,
        geom_late=gdf_SARIn_3_6.geometry)
).compute()

In [ ]:
# Clip temporally as to not overlap with ICESat-2 data

# Remove time slices that occur during the ICESat-2 era that will not be used 
# to conserve memory when loaded for data analysis

# end_date includes one quarter of overlapping data with ICESat-2 time series
# to allow for cyc-to-cyc differencing to remove datum from delta_h to create cycle-to-cycle dh
# end_date = '2019-01-01T00:00:00.000000000' # Extra time step required if doing cycle-to-cycle dh
end_date = '2018-10-01T18:00:00.000000000'

CS2_h_clipped = CS2_h_clipped.sel(time=slice(None, end_date))

# Preview temporally subset data set's time variable
CS2_h_clipped['time']

In [ ]:
# Reindex the smaller one onto the larger grid
CS2_on_IS2 = CS2_h_clipped.reindex(y=IS2_h.y, x=IS2_h.x)


## Multi-mission absolute height time series

In [ ]:

# 2) Concatenate along time
CS2_IS2_h = xr.concat([CS2_on_IS2, IS2_h], dim="time")

# # 3) Optional: sort by time
# CS2_IS2_h = CS2_IS2_h.sortby("time")

# View dataset
CS2_IS2_h['time']

In [ ]:
CS2_IS2_h

In [ ]:
# plt.close()
# CS2_IS2_h[:,:,-1].plot()
# plt.show()

# Data analysis
Use Shreve (1972) equation to calculate hydropotential using bed elevations and ice-surface elevations (less firn depth to get ice mass).

In [ ]:
# # Step 1: Compute hydropotential

# # Use BedMachine bed topography for Zbed and surface topography (less firn) for Zsurf

# # Define densities (rho) of ice and water
# rho_ice = 917
# rho_water = 997

# # Calculate hydropotential using Shreve, 1972 equation
# dynamic_subglacial_hydropotential = (9.8 * ((rho_ice*(bedmachine['surface']-bedmachine['firn'])) + (rho_water-rho_ice)*bedmachine['bed'])) / 1e3

# # Display xarray.Dataset metadata
# dynamic_subglacial_hydropotential

In [ ]:
# # Step 2: Mask to grounded ice only

# # Create a mask where values == 2 (grounded ice)
# grounded_ice_mask = (bedmachine['mask'] == 2)

# # Apply the mask to dynamic_subglacial_hydropotential
# dynamic_subglacial_hydropotential = dynamic_subglacial_hydropotential.where(grounded_ice_mask)

# # Display xarray.Dataset metadata to ensure edges have become nan's
# dynamic_subglacial_hydropotential

In [ ]:
# # Step 1: Compute hydropotential

# Assign new data variable of hydropotential to ATL15 xarray.Dataset
# Follows Shreve 1972, where h_i=Zsurf-Zbed
# Adds g (gravitational acceleration) to make units Pa
# Use BedMachine Antarctica bed topography for Zbed
rho_ice = 917
rho_water = 997
dynamic_subglacial_hydropotential = (9.8 * (rho_ice*(CS2_IS2_h-bedmachine['firn']) + (rho_water-rho_ice)*bedmachine['bed']))

In [ ]:
# View newly created datset
dynamic_subglacial_hydropotential

In [ ]:
# def get_data_extent_with_buffer(da, buffer_percent=0.05):
#     """
#     Find the extent of non-NaN data and add a buffer.
    
#     Parameters:
#     -----------
#     da : xarray.DataArray
#         The data array to analyze
#     buffer_percent : float
#         Buffer as a percentage of the data range (default 5%)
    
#     Returns:
#     --------
#     tuple : (x_min, x_max, y_min, y_max) with buffer applied
#     """
#     # Find where data is not NaN
#     valid_data = ~np.isnan(da)
    
#     # Get coordinates where we have valid data
#     y_coords, x_coords = np.where(valid_data)
    
#     if len(x_coords) == 0 or len(y_coords) == 0:
#         print("Warning: No valid data found!")
#         return None
    
#     # Convert array indices to actual coordinate values
#     x_values = da.coords[da.dims[1]].values  # assuming dims are [y, x]
#     y_values = da.coords[da.dims[0]].values

#     # Get the actual coordinate bounds where we have data
#     x_data_min = x_values.min()
#     x_data_max = x_values.max()
#     y_data_min = y_values.min()
#     y_data_max = y_values.max()
    
#     # Calculate buffer
#     x_range = x_data_max - x_data_min
#     y_range = y_data_max - y_data_min
    
#     x_buffer = x_range * buffer_percent
#     y_buffer = y_range * buffer_percent
    
#     # Apply buffer
#     x_min = x_data_min - x_buffer
#     x_max = x_data_max + x_buffer
#     y_min = y_data_min - y_buffer
#     y_max = y_data_max + y_buffer
    
#     return x_min, x_max, y_min, y_max

# # Get the data extent with 5% buffer
# extent = get_data_extent_with_buffer(dynamic_subglacial_hydropotential[:,:,0], buffer_percent=0.01)

# if extent is not None:
#     x_min, x_max, y_min, y_max = extent
    
#     # Plot results and save figure
#     fig, ax = plt.subplots(figsize=(10, 8))

#     # Use actual data range for asymmetric colorbar
#     data_min = float(dynamic_subglacial_hydropotential.min())
#     data_max = float(dynamic_subglacial_hydropotential.max())

#     # Create the plot
#     im = dynamic_subglacial_hydropotential[:,:,-1].plot(
#         ax=ax, 
#         add_colorbar=False,
#         cmap='viridis',
#         vmin=data_min,
#         vmax=data_max
#     )

#     # Set the axis limits to clip to data extent + buffer
#     ax.set_xlim(x_min, x_max)
#     ax.set_ylim(y_min, y_max)
    
#     # Format axes
#     ax.set_aspect('equal')
#     ax.set_axis_off()

#     # Create colorbar axes with same width as plot
#     divider = make_axes_locatable(ax)
#     cax = divider.append_axes('bottom', size='3%', pad=0.01)
    
#     # Add colorbar with label
#     cbar = plt.colorbar(im, cax=cax, orientation='horizontal')
#     cbar.set_label('subglacial hydropotential [kPa]')
#     cbar.ax.tick_params(labelrotation=45)
    
#     plt.tight_layout()
#     plt.show()

#     plt.savefig('output/subglacial_hydropotential_Antarctica.png', bbox_inches='tight')

# Export hydropotental dataset

In [ ]:
# Prepare dataset for export

# Add variable metadata
dynamic_subglacial_hydropotential.attrs = {
    'units': 'Pa',
    'ice_density' : 917.0,
    'ice_density_units': 'kg m-3',
    'freshwater_density' : 997.0,
    'freshwater_density_units': 'kg m-3',
}

# Convert to dataset for chucking with Zarr
ds = dynamic_subglacial_hydropotential.to_dataset(name='dynamic_subglacial_hydropotential')

# Add global metadata
ds.attrs = {
    'conventions': 'CF-1.8',
    'title': 'Dynamic Antarctic subglacial hydropotential',
    'description': 'Dynamic Antarctic subglacial gridded hydropotential calculated using Shreve (1972) equation \
        using BedMachine Antarctica v3 bed topography and firn air content (Morlighem et al., 2020; Morlighem, 2022) \
        and ice surface heights from CryoSat-2 SARIn mode (Smith & Sauthoff, 2025) and ICESat-2 ATL14/15 (Smith and others, 2024a,b).',
    'history': 'Created 2025-07-20',
    'identifier_product_DOI': 'doi:10.5281/zenodo.16243278',
    'citation': 'Sauthoff, W. & Siegfried, M. R. (2025). Antarctic subglacial hydropotential [Data set]. Zenodo. https://doi.org/10.5281/zenodo.16243278',
    'license': 'CC BY-SA 4.0',
    'region': 'Antarctica'
}

# Copy mapping variable into new dataset as variable with zero dimensions
mapping_attrs = bedmachine['mapping'].attrs
ds['mapping'] = xr.DataArray(np.array(0, dtype=np.int32), attrs=mapping_attrs)

# Link the mapping variable to hydropotential variable
ds['dynamic_subglacial_hydropotential'].attrs['grid_mapping'] = 'mapping'

# Copy global CRS-related attributes
ds.attrs['proj4'] = bedmachine.attrs['proj4']
ds.attrs['Projection'] = bedmachine.attrs['Projection']

# Add crs and crs_wkt
ds.attrs['crs'] = 'EPSG:3031'
crs = CRS.from_epsg(3031)
ds.attrs['crs_wkt'] = crs.to_wkt()

# View ds metadata
ds

## Export to chucked netcdf

In [ ]:
# Export with chunking + compression
ds.to_netcdf(
    'output/dynamic_subglacial_hydropotential_Antarctica.nc',
    encoding={
        'dynamic_subglacial_hydropotential': {
            'dtype': 'float32',
            'zlib': True,
            'complevel': 4,
            'chunksizes': (500, 500, 1)  # (y, x, time)
        }
    }
)

## Validate dataset after export
First check climate and forecast conventions, then importing exported file, viewing metadata and plotting data.

In [ ]:
!conda install -c conda-forge cfchecker --quiet -y

In [ ]:
!cfchecks output/dynamic_subglacial_hydropotential_Antarctica.nc

In [ ]:
ds_imported = xr.open_dataset('output/dynamic_subglacial_hydropotential_Antarctica.nc')
ds_imported

In [ ]:
ds_imported.info()

In [ ]:
# Check that NaN values are preserved
print('Original NaN count:', ds['dynamic_subglacial_hydropotential'].isnull().sum())
print('Imported NaN count:', ds_imported['dynamic_subglacial_hydropotential'].isnull().sum())

In [ ]:
plt.figure()
ds_imported['dynamic_subglacial_hydropotential'][:,:,-1].plot()
plt.show()

In [ ]:
# Opened netcdf in Panoply to ensure compatibility

## Export to Zarr

In [ ]:
# Chunk for performance (tune based on size and use case)
ds = ds.chunk({'x': 500, 'y': 500})

# Write to a Zarr store
ds.to_zarr('output/dynamic_subglacial_hydropotential_Antarctica.zarr', mode='w',  consolidated=True)

In [ ]:
# Zip zarr files for upload to Zenodo
shutil.make_archive(
    'output/dynamic_subglacial_hydropotential_Antarctica.zarr',  # output path (no zip extension)
    'zip',
    'output/dynamic_subglacial_hydropotential_Antarctica.zarr'   # source
)

In [ ]:
!rm -r 'output/dynamic_subglacial_hydropotential_Antarctica.zarr'

## Validate dataset after export
Unzip zip file of Zarr store, then import data, view metadata, plot data.

In [ ]:
# Extract the zip file
with zipfile.ZipFile('output/dynamic_subglacial_hydropotential_Antarctica.zarr.zip', 'r') as zip_ref:
    zip_ref.extractall('output/dynamic_subglacial_hydropotential_Antarctica.zarr')

# Now open normally
ds_imported = xr.open_zarr('output/dynamic_subglacial_hydropotential_Antarctica.zarr', consolidated=True)
ds_imported

In [ ]:
ds_imported.info()

In [ ]:
# Check that NaN values are preserved
print('Original NaN count:', ds['subglacial_hydropotential'].isnull().sum().compute())
print('Imported NaN count:', ds_imported['subglacial_hydropotential'].isnull().sum().compute())

In [ ]:
plt.figure()
ds_imported['subglacial_hydropotential'].plot()
plt.show()

In [ ]:
# Remove Zarr store folder
!rm -r 'output/dynamic_subglacial_hydropotential_Antarctica.zarr'

# Remove temporary files
Files are first downloaded locally to upload to Zenodo repo.

In [ ]:
!rm 'output/dynamic_subglacial_hydropotential_Antarctica.nc'

In [ ]:
!rm 'output/dynamic_subglacial_hydropotential_Antarctica.zarr.zip'

# References

Morlighem, M., Rignot, E., Binder, T., Blankenship, D., Drews, R., Eagles, G., et al. (2020). Deep glacial troughs and stabilizing ridges unveiled beneath the margins of the Antarctic ice sheet. _Nature Geoscience_, 13(2), 132–137. [doi:10.1038/s41561-019-0510-8](https://doi.org/10.1038/s41561-019-0510-8)

Morlighem, M. (2022). MEaSUREs BedMachine Antarctica. (NSIDC-0756, Version 3). [Data Set]. Boulder, Colorado USA. NASA National Snow and Ice Data Center Distributed Active Archive Center. https://doi.org/10.5067/FPSU0V1MWUB6. Date Accessed 05-19-2025.

Sauthoff, W., & Siegfried, M. R. (2025). Antarctic subglacial hydropotential [Data set]. Zenodo. [https://doi.org/10.5281/zenodo.16243278](10.5281/zenodo.16243278)

Smith, B., & Sauthoff, W. (2025). CryoSat-2 SARIn Height Change and Reference DEM for Antarctica (1.0) [Data set]. Zenodo. [https://doi.org/10.5281/zenodo.14963551](https://doi.org/10.5281/zenodo.14963551)

Smith, B., Sutterley, T., Dickinson, S., Jelley, B. P., Felikson, D., Neumann, T. A., Fricker, H. A., Gardner, A. S., Padman, L., Markus, T., Kurtz, N., Bhardwaj, S., Hancock, D. & Lee, J. (2024). ATLAS/ICESat-2 L3B Gridded Antarctic and Arctic Land Ice Height. (ATL14, Version 4). [Data Set]. Boulder, Colorado USA. NASA National Snow and Ice Data Center Distributed Active Archive Center. https://doi.org/10.5067/ATLAS/ATL14.004. Date Accessed 12-01-2025.

Smith, B., Sutterley, T., Dickinson, S., Jelley, B. P., Felikson, D., Neumann, T. A., Fricker, H. A., Gardner, A. S., Padman, L., Markus, T., Kurtz, N., Bhardwaj, S., Hancock, D. & Lee, J. (2024). ATLAS/ICESat-2 L3B Gridded Antarctic and Arctic Land Ice Height Change. (ATL15, Version 4). [Data Set]. Boulder, Colorado USA. NASA National Snow and Ice Data Center Distributed Active Archive Center. https://doi.org/10.5067/ATLAS/ATL15.004. Date Accessed 12-01-2025.

Shreve, R. L. (1972). Movement of Water in Glaciers. _Journal of Glaciology_, 11(62), 205–214. [doi:10.3189/S002214300002219X](https://doi.org/10.3189/S002214300002219X)